In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install openpyxl

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import regularizers
from keras import models
from openpyxl import Workbook

In [ ]:
path_data = "/content/drive/MyDrive/SuccKD_MultiSpecies/Data/"
path_model = "/content/drive/MyDrive/SuccKD_MultiSpecies/Model/"
path_result = "/content/drive/MyDrive/SuccKD_MultiSpecies/Result_cv/"

In [ ]:
file_train = "Train_S.lycopersicum_succ.csv"

In [ ]:
# Build an amino acid dictionary
def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)


In [ ]:
#  Load train data and pre-processing
df_train =pd.read_csv(path_data+file_train, delimiter= ',')
texts_train =[]
for i in df_train['Peptide_WS_33']:
  temp = ProSentence(i,k)
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)
# Tokenizer
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())
MAX_SEQUENCE_LENGTH = len(data_token[1])
Xtrain = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = np.array(df_train['Label'])
ytrain = np.array(ytrain)
#one-hot encoding on the labels
lb = LabelBinarizer()
ytrain= lb.fit_transform(ytrain)
ytrain = to_categorical(ytrain)
ytrain.shape


(726, 2)

In [ ]:
TIME_STEPS = MAX_SEQUENCE_LENGTH
INPUT_SIZE = 300

In [ ]:
# KD2_succi
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras import models, layers, losses

# Teacher 1
def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.4))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model

# Teacher 2
def create_bilstm_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
# Student
def create_student():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.2))
    model.add(layers.BatchNormalization())
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
#  KD2_succi: Knowlege distillation
import tensorflow as tf
from tensorflow.keras import optimizers

class Distiller(tf.keras.Model):
    def __init__(self, teacher1, teacher2, student, alpha=0.5, temperature=3.0):
        super(Distiller, self).__init__()
        self.teacher1 = teacher1
        self.teacher2 = teacher2
        self.student = student
        self.alpha = alpha
        self.temperature = temperature

        # Optimizer for the student
        self.student_optimizer = AdamW(learning_rate=0.0001)

    def compile(self, metrics, student_loss_fn, distillation_loss_fn):
        super(Distiller, self).compile(metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn

    def call(self, inputs):
        """
        This method defines how the model processes inputs.
        Args:
        - inputs: Input data (Xtest_student or any other data).

        Returns:
        - Output logits from the student model.
        """
        student_logits = self.student(inputs, training=False)
        return student_logits

    def train_step(self, data):
        x, y = data

        # Sequentially forward pass through teachers
        teacher1_logits = self.teacher1(x, training=False)
        teacher2_logits = self.teacher2(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass through student
            student_logits = self.student(x, training=True)

            # Compute the task loss (for hard labels)
            student_loss = self.student_loss_fn(y, student_logits)

            # Compute distillation losses (for soft labels)
            distill_loss1 = self.distillation_loss_fn(
                tf.nn.softmax(teacher1_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            distill_loss2 = self.distillation_loss_fn(
                tf.nn.softmax(teacher2_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            # Combined loss
            total_loss = self.alpha * student_loss + (1 - self.alpha) * (distill_loss1 + distill_loss2)

        # Compute gradients and update student model
        grads = tape.gradient(total_loss, self.student.trainable_variables)
        self.student_optimizer.apply_gradients(zip(grads, self.student.trainable_variables))

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        # Return results
        results = {
            'student_loss': student_loss,
            'distill_loss1': distill_loss1,
            'distill_loss2': distill_loss2
        }
        results.update({m.name: m.result() for m in self.metrics})
        return results

    def test_step(self, data):
        x, y = data
        student_logits = self.student(x, training=False)

        # Compute the task loss (for hard labels)
        student_loss = self.student_loss_fn(y, student_logits)

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        return {
            'student_loss': student_loss,
            **{m.name: m.result() for m in self.metrics}
        }



In [ ]:
# Transfer learning from Generic dl set to species
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW
from keras import models
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []
confusion_matrices = []

# K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
    student_fold = create_student()
    teacher1_fold = models.load_model(path_model +"model_teacher21.h5") # Sử dụng teacher giống nhau trên cùng loài
    teacher2_fold = models.load_model(path_model +"model_teacher22.h5")
    distiller_fold = Distiller(teacher1=teacher1_fold, teacher2=teacher2_fold, student=student_fold, alpha=0.5, temperature=10)
    distiller_fold.compile(
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),

        metrics=["categorical_accuracy", "AUC"],
        distillation_loss_fn=keras.losses.KLDivergence(),
    )
    # Train model
    history = distiller_fold.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=16,
        epochs=50,
    )

    results = distiller_fold.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)
    y_pred_fold = student_fold.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)

    # confusion matrix and TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Average the results from the folds
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(distiller_fold.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")

#  Show TP, TN, FP, FN
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# save TP, TN, FP, FN into file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_S.lycopersicum_KD2_Succi.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_S.lycopersicum_KD2_Succi.xlsx'")

Training on fold 1/10...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


41/41 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - AUC: 0.3642 - categorical_accuracy: 0.4086 - distill_loss1: 0.0371 - distill_loss2: 0.0837 - loss: 0.5000 - student_loss: 0.7161 - val_AUC: 0.5887 - val_categorical_accuracy: 0.6027 - val_loss: 0.5000 - val_student_loss: 0.6673
Epoch 2/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - AUC: 0.6171 - categorical_accuracy: 0.5818 - distill_loss1: 0.0319 - distill_loss2: 0.0736 - loss: 0.5000 - student_loss: 0.6729 - val_AUC: 0.6041 - val_categorical_accuracy: 0.6027 - val_loss: 0.5000 - val_student_loss: 0.6414
Epoch 3/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - AUC: 0.7056 - categorical_accuracy: 0.6575 - distill_loss1: 0.0280 - distill_loss2: 0.0674 - loss: 0.5000 - student_loss: 0.6347 - val_AUC: 0.6281 - val_categorical_accuracy: 0.6027 - val_loss: 0.5000 - val_student_loss: 0.6121
Epoch 4/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - AUC: 0.7542 - categorical_accuracy: 0.6856 - distill_loss1: 0.0264 - distill_loss2: 0.0630 - loss: 0.5000 - studen

...

41/41 ━━━━━━━━━━━━━━━━━━━━ 14s 70ms/step - AUC: 0.4984 - categorical_accuracy: 0.4938 - distill_loss1: 0.0343 - distill_loss2: 0.0792 - loss: 0.5000 - student_loss: 0.6939 - val_AUC: 0.6549 - val_categorical_accuracy: 0.6111 - val_loss: 0.5000 - val_student_loss: 0.6564
Epoch 2/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - AUC: 0.6480 - categorical_accuracy: 0.5974 - distill_loss1: 0.0304 - distill_loss2: 0.0717 - loss: 0.5000 - student_loss: 0.6553 - val_AUC: 0.6906 - val_categorical_accuracy: 0.6111 - val_loss: 0.5000 - val_student_loss: 0.6301
Epoch 3/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - AUC: 0.7268 - categorical_accuracy: 0.6711 - distill_loss1: 0.0281 - distill_loss2: 0.0670 - loss: 0.5000 - student_loss: 0.6256 - val_AUC: 0.7010 - val_categorical_accuracy: 0.6111 - val_loss: 0.5000 - val_student_loss: 0.5924
Epoch 4/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - AUC: 0.7857 - categorical_accuracy: 0.7114 - distill_loss1: 0.0266 - distill_loss2: 0.0619 - loss: 0.5000 - studen

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


41/41 ━━━━━━━━━━━━━━━━━━━━ 17s 81ms/step - AUC: 0.6199 - categorical_accuracy: 0.6047 - distill_loss1: 0.0307 - distill_loss2: 0.0727 - loss: 0.5000 - student_loss: 0.6666 - val_AUC: 0.6950 - val_categorical_accuracy: 0.6806 - val_loss: 0.5000 - val_student_loss: 0.6678
Epoch 2/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - AUC: 0.7224 - categorical_accuracy: 0.6601 - distill_loss1: 0.0284 - distill_loss2: 0.0674 - loss: 0.5000 - student_loss: 0.6319 - val_AUC: 0.7313 - val_categorical_accuracy: 0.6806 - val_loss: 0.5000 - val_student_loss: 0.6423
Epoch 3/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - AUC: 0.7427 - categorical_accuracy: 0.6812 - distill_loss1: 0.0278 - distill_loss2: 0.0632 - loss: 0.5000 - student_loss: 0.6071 - val_AUC: 0.7595 - val_categorical_accuracy: 0.6806 - val_loss: 0.5000 - val_student_loss: 0.6181
Epoch 4/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - AUC: 0.7404 - categorical_accuracy: 0.6779 - distill_loss1: 0.0275 - distill_loss2: 0.0617 - loss: 0.5000 - studen

In [ ]:
# Train dataset with S.lycopersicum
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []
confusion_matrices = []
# K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
    teacher1_fold = create_cnn_model()
    teacher2_fold = create_bilstm_model()
    student_fold = create_student()
    teacher1_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher2_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher1_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )

    teacher2_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )

    # Re-initialize the distiller for each foldd
    distiller_fold = Distiller(teacher1=teacher1_fold, teacher2=teacher2_fold, student=student_fold, alpha=0.5, temperature=10)
    distiller_fold.compile(
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),

        metrics=["categorical_accuracy", "AUC"],
        distillation_loss_fn=keras.losses.KLDivergence(),
    )

    # Train model
    history = distiller_fold.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=16,
        epochs=50,
        verbose=1
    )

    # Save result of each fold
    results = distiller_fold.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)

    # Dự đoán nhãn cho tập validation
    y_pred_fold = student_fold.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)

    # confusion matrix and save TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Average the results from the folds
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(distiller_fold.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")

# Show TP, TN, FP, FN for each fold
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# save TP, TN, FP, FN into the file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_S.lycopersicum_KD2_Succ1.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_S.lycopersicum_KD2_Succ1.xlsx'")

Training on fold 1/10...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


41/41 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - AUC: 0.5852 - categorical_accuracy: 0.5756 - loss: 0.7384
Epoch 2/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - AUC: 0.6349 - categorical_accuracy: 0.5953 - loss: 0.7147
Epoch 3/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - AUC: 0.6560 - categorical_accuracy: 0.6200 - loss: 0.6995
Epoch 4/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - AUC: 0.6155 - categorical_accuracy: 0.5889 - loss: 0.7180
Epoch 5/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - AUC: 0.5869 - categorical_accuracy: 0.5737 - loss: 0.7685
Epoch 6/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - AUC: 0.6716 - categorical_accuracy: 0.6309 - loss: 0.6872
Epoch 7/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - AUC: 0.7124 - categorical_accuracy: 0.6675 - loss: 0.6313
Epoch 8/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - AUC: 0.6518 - categorical_accuracy: 0.6271 - loss: 0.6848
Epoch 9/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - AUC: 0.6794 - categorical_accuracy: 0.6333 - loss: 0.6606
Epoch 10/50

...

41/41 ━━━━━━━━━━━━━━━━━━━━ 12s 75ms/step - AUC: 0.5817 - categorical_accuracy: 0.5487 - distill_loss1: 0.0116 - distill_loss2: 0.1137 - loss: 0.5000 - student_loss: 0.6773 - val_AUC: 0.6229 - val_categorical_accuracy: 0.6111 - val_loss: 0.5000 - val_student_loss: 0.6743
Epoch 2/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - AUC: 0.6898 - categorical_accuracy: 0.6324 - distill_loss1: 0.0088 - distill_loss2: 0.1030 - loss: 0.5000 - student_loss: 0.6331 - val_AUC: 0.6783 - val_categorical_accuracy: 0.6111 - val_loss: 0.5000 - val_student_loss: 0.6357
Epoch 3/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - AUC: 0.7615 - categorical_accuracy: 0.6899 - distill_loss1: 0.0077 - distill_loss2: 0.0968 - loss: 0.5000 - student_loss: 0.6077 - val_AUC: 0.7058 - val_categorical_accuracy: 0.6111 - val_loss: 0.5000 - val_student_loss: 0.5952
Epoch 4/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - AUC: 0.8148 - categorical_accuracy: 0.7384 - distill_loss1: 0.0070 - distill_loss2: 0.0877 - loss: 0.5000 - studen

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


41/41 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - AUC: 0.3345 - categorical_accuracy: 0.3627 - loss: 1.2114
Epoch 2/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - AUC: 0.5567 - categorical_accuracy: 0.5697 - loss: 0.7968
Epoch 3/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - AUC: 0.6018 - categorical_accuracy: 0.5888 - loss: 0.7642
Epoch 4/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - AUC: 0.6171 - categorical_accuracy: 0.5915 - loss: 0.7436
Epoch 5/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - AUC: 0.6260 - categorical_accuracy: 0.5854 - loss: 0.7281
Epoch 6/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - AUC: 0.6514 - categorical_accuracy: 0.5984 - loss: 0.6967
Epoch 7/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - AUC: 0.6362 - categorical_accuracy: 0.6061 - loss: 0.7146
Epoch 8/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - AUC: 0.6213 - categorical_accuracy: 0.6077 - loss: 0.7182
Epoch 9/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - AUC: 0.6345 - categorical_accuracy: 0.5971 - loss: 0.7037
Epoch 10/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


41/41 ━━━━━━━━━━━━━━━━━━━━ 14s 99ms/step - AUC: 0.6617 - categorical_accuracy: 0.6134 - distill_loss1: 0.0112 - distill_loss2: 0.1095 - loss: 0.5000 - student_loss: 0.6567 - val_AUC: 0.6806 - val_categorical_accuracy: 0.6806 - val_loss: 0.5000 - val_student_loss: 0.6629
Epoch 2/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - AUC: 0.7161 - categorical_accuracy: 0.6791 - distill_loss1: 0.0084 - distill_loss2: 0.1017 - loss: 0.5000 - student_loss: 0.6233 - val_AUC: 0.7508 - val_categorical_accuracy: 0.6806 - val_loss: 0.5000 - val_student_loss: 0.6390
Epoch 3/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - AUC: 0.7271 - categorical_accuracy: 0.6793 - distill_loss1: 0.0071 - distill_loss2: 0.0991 - loss: 0.5000 - student_loss: 0.6112 - val_AUC: 0.7749 - val_categorical_accuracy: 0.6806 - val_loss: 0.5000 - val_student_loss: 0.6117
Epoch 4/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - AUC: 0.7871 - categorical_accuracy: 0.7113 - distill_loss1: 0.0063 - distill_loss2: 0.0943 - loss: 0.5000 - studen